# End of Semester Grade Calculator

In [ ]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
from helpers import read_gradebook_csvs, assignment_titles, component_titles, to_percentages, component_grade, component_grades

Multiple gradescope gradebooks will be merged into a single DataFrame.

Add the files to the list below in reverse chronological order of the final exam open time. Doing so will ensure that if a student opened / viewed an earlier final and a later final (e.g. makeup or INTL) the earlier will count. This also allows for the merging of alternate Gradescopes (e.g. for late submissions). The first non-zero score for a given entry takes highest precedence if there are multiple entries for the same student across many CSVs.

In [ ]:
gradebook_csv_files = [
    "./COMP110_Spring_2022_grades.csv",
]

grades = read_gradebook_csvs(gradebook_csv_files)
print(f"{grades.shape[0]} records read")
grades.head(5)

## Merge in PIDs, ONYENs, and Sections

Follow through the script in `../roster.ipynb` to get a `roster_merged.csv` file that can serve as a mapping between Connect Carolina, Sakai, and Gradesope.

In [ ]:
roster = pd.read_csv("./roster_merged.csv")
roster = roster.set_index("GS Email")
roster.head(5)

Where there are students in the official roster, but not in the gradebook, add their correct e-mail mapping to email_mappings.

In [ ]:
# Loop through all e-mails in CC roster
# Find their entry in grades
# Associate their grade DF row with their PID and section
for email in roster.index:
    try:
        student = grades.loc[email]
    except KeyError:
        print(f"Could not find {email} in grades - add to mappings")
        continue
    grades.loc[email, ["Name"]] = roster.loc[email]["Name"]
    grades.loc[email, ["PID"]] = roster.loc[email]["PID"]
    grades.loc[email, ["ONYEN"]] = roster.loc[email]["ONYEN"]
    grades.loc[email, ["First"]] = roster.loc[email]["First"]
    grades.loc[email, ["Last"]] = roster.loc[email]["Last"]
    grades.loc[email, ["Section"]] = roster.loc[email]["Section"]

# Drop people no longer in an official ConnectCarolina section
grades = grades[~grades["Section"].isnull()].copy()

# Convert PIDs to ints
grades["PID"] = grades["PID"].astype(int)

grades.to_csv("gradescope-with-onyens.csv")

grades

## Set readings Max Points to 100

Readings 00 and 02 were graded strangely on GS, where the max points is listed as 0.0, so we need to do a bit of preprossing.

In [ ]:
grades["RD00 - The Ethical Algorithm - Max Points"] = 100.0
grades["RD01 - WOMD - Max Points"] = 100.0
grades["RD02 - Gender Shades - Max Points"] = 100.0

## Get Percentages for All Assignments

In [ ]:
grades_pct = to_percentages(grades)
grades_pct.to_csv("grades.csv")
grades_pct

## Handle Final Grade Replacement Logic

We had 5 quizzes.  If the final is lower than a quiz score, it will replace a single quiz score.

In [ ]:
grades_copy = grades_pct.copy()

quizzes = component_titles(grades_copy.columns, "QZ")
quiz_sum = grades_copy[quizzes].sum(axis=1)
quiz_low = grades_copy[quizzes].min(axis=1)
final_score = grades_pct[["FN00"]]["FN00"]

grades_pct.loc[final_score > quiz_low, "QU_SCORE"] = (quiz_sum - quiz_low + final_score) / 4.0
grades_pct.loc[final_score <= quiz_low, "QU_SCORE"] = quiz_sum / 4.0
grades_pct["REPLACED_QUIZ"] = final_score > quiz_low
grades_pct.to_csv("replace_quiz.csv")
grades_pct

## Compute the Grades for Each Component of the Course

In [ ]:
components = {
    "Final": {
        "prefix": "FN",
        "drops": 0,
        "weight": 10.0
    },
    "Quizzes": {
        "prefix": "QZ",
        "drops": 0,
        "weight": 40.0
    },
    "Readings": {
        "prefix": "RD",
        "drops": 0,
        "weight": 10.0
    },
    "Lessons": {
        "prefix": "LS",
        "drops": 3,
        "weight": 10.0
    },
    "Exercises": {
        "prefix": "EX",
        "drops": 1,
        "weight": 30.0
    }
}

grades_parts = component_grades(grades_pct, components)
grades_parts = grades_parts.sort_values(["Section", "Total"], ascending=[True, True])
grades_parts.to_csv("components.csv")
grades_parts.tail()

In [ ]:
cutoffs = {
    "F": -0.1,
    "D": 59.0,
    "D+": 66.0,
    "C-": 69.0,
    "C": 72.0,
    "C+": 76.0,
    "B-": 79.0,
    "B": 82.0,
    "B+": 86.0,
    "A-": 89.0,
    "A": 92.0 
}

for letter in cutoffs:
    grades_parts.loc[grades_parts["Total"] >= cutoffs[letter], "Grade"] = letter

grades_parts.loc[(grades_parts["FN"] == 0.0) & (grades_parts["Total"] < 55.0), "Grade"] = "FA"
grades_parts.loc[(grades_parts["FN"] == 0.0) & (grades_parts["Total"] >= 55.0), "Grade"] = "AB"
grades_parts.to_csv("grades.csv")
grades_parts

## Produce CSVs for 

In [ ]:
grades_parts["Final Exam"] = grades_parts["FN"] * 100.0
grades_parts["Final Exam"] = grades_parts["Final Exam"].round(0)
grades_parts["Course Grade"] = grades_parts["Grade"]

for section in [1, 2]:
    grades_sakai = grades_parts[(grades_parts["Section"] == section) & (grades_parts["Grade"] != "FA")][["ONYEN", "Final Exam", "Course Grade"]]
    grades_sakai.to_csv(f"s{section}-sakai.csv", header=["ID", "Final Exam Score", "Course Grade"], index=False)
    grades_cc = grades_parts[grades_parts["Section"] == section][["PID", "ONYEN", "ONYEN", "Course Grade"]]
    grades_cc.to_csv(f"s{section}-connect-carolina.csv", header=False, index=False)

In [ ]:
grades_parts[grades_parts["Total"] >= 86.0].shape[0] / grades_parts.shape[0]